In [19]:
import os
import json
from dotenv import load_dotenv
import telebot
from time import sleep

from langchain_community.chat_models import GigaChat
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

In [20]:
load_dotenv('config.env')

credentials = os.getenv('GIGACHAT_CREDENTIALS')
tbot_token = os.getenv('TELEGRAM_TOKEN')

In [21]:
bot = telebot.TeleBot(tbot_token)

## Подключение LLM

In [22]:
llm = GigaChat(credentials=credentials, verify_ssl_certs=False)
conversation = ConversationChain(llm=llm,
                                 verbose=True,
                                 memory=ConversationBufferMemory())

# доп
# system_prompt = PromptTemplate.from_template()

## Память

In [23]:
# хранение пользователя и его ConversationBufferMemory
user_conversations = {}

In [24]:
MEMORY_FILE = 'user_session.json'

def load_memory():
    with open(MEMORY_FILE, "r", encoding="utf-8") as f:
        data = json.load(f)
    for uid, history in data.items():
        mem = ConversationBufferMemory()
        mem.chat_memory.add_user_message(history.get("user", ""))
        mem.chat_memory.add_ai_message(history.get("bot", ""))
        user_conversations[int(uid)] = ConversationChain(
            llm=llm, memory=mem, 
            # prompt=system_prompt, 
            verbose=False
        )

def save_memory():
    data = {}
    for uid, conv in user_conversations.items():
        messages = conv.memory.chat_memory.messages
        if len(messages) >= 2:
            data[uid] = {
                "user": messages[-2].content,
                "bot": messages[-1].content,
            }
    with open(MEMORY_FILE, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

## Обработчик сообщений

In [25]:
@bot.message_handler(content_types=['text'])
def handle_text_message(message):
    user_id = message.chat.id
    text = message.text.strip()

    if user_id not in user_conversations:
        user_conversations[user_id] = ConversationChain(
            llm=llm,
            memory=ConversationBufferMemory(),
            verbose=False
        )

    conversation = user_conversations[user_id]

    response = conversation.predict(input=text)
    bot.send_message(user_id, response)
    save_memory()

In [26]:
bot.polling(none_stop=True)

# webhook

ReadTimeout: HTTPSConnectionPool(host='api.telegram.org', port=443): Read timed out. (read timeout=30)